In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np

In [0]:
data = pd.read_csv("/content/drive/My Drive/toxicity/train.csv")

In [4]:
print(data.shape)

(1804874, 45)


In [5]:
data = data[['target', 'comment_text']] 
print(data.shape)

(1804874, 2)


In [0]:
data.replace('', np.nan, inplace=True)

In [0]:
data.dropna( inplace=True)

In [8]:
print(data.shape)

(1804874, 2)


In [9]:
print(data.head())

     target                                       comment_text
0  0.000000  This is so cool. It's like, 'would you want yo...
1  0.000000  Thank you!! This would make my life a lot less...
2  0.000000  This is such an urgent design problem; kudos t...
3  0.000000  Is this something I'll be able to install on m...
4  0.893617               haha you guys are a bunch of losers.


In [10]:
data.target  = pd.to_numeric(data.target , errors='coerce', downcast='float')
data.dropna( inplace=True)
print(data.shape)

(1804874, 2)


In [11]:
pos = data[data["target"]>=0.5]
print(pos.shape)

(144334, 2)


In [12]:
neg = data[data["target"]<0.5]
print(neg.shape)

(1660540, 2)


In [13]:
neg = data.sample(n=pos.shape[0],replace=True)
print(neg.shape)

(144334, 2)


In [14]:
data = pd.concat([pos, neg])
print(data.shape)

(288668, 2)


In [15]:
print(data.head())

      target                                       comment_text
4   0.893617               haha you guys are a bunch of losers.
5   0.666667                               ur a sh*tty comment.
13  0.600000  It's ridiculous that these guys are being call...
14  0.500000  This story gets more ridiculous by the hour! A...
19  0.500000  Angry trolls, misogynists and Racists", oh my....


In [16]:
data.target = np.where(data["target"]>=0.5, 1,0)
print(data.shape)


(288668, 2)


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(np.array(data.comment_text),
                                                    np.array(data.target), test_size=0.2, 
                                                    random_state=42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(230934,) (230934,)
(57734,) (57734,)


In [18]:
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

Using TensorFlow backend.


In [0]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('/content/drive/My Drive/toxicity/glove.6B.100d.txt')

In [21]:
word = "0"
print(word_to_index[word])

1681


In [22]:
import string
max_len = 0
avg_len = 0
num = 0
for i in data.comment_text:
  tmp = len(i.translate(str.maketrans('','',string.punctuation)).lower().split())
  num+=1
  avg_len += tmp
  if tmp>max_len:
    max_len = tmp
avg_len = avg_len/num
print(max_len)
print(avg_len)

311
49.47640542076018


In [0]:
def sentences_to_indices(X, word_to_index, max_len):
    m = X.shape[0]                                
    X_indices = np.zeros((m, max_len))
    
    for i in range(m): 
        #print(X[i])
        tmp = X[i].translate(str.maketrans('','',string.punctuation)).lower().split()
        j = 0
        for w in tmp:
          if j>= max_len:
            break
          try:
            X_indices[i, j] = word_to_index[w]
            j = j+1
            #print(w)
          except KeyError:
            #print(w)
            pass
            
    
    return X_indices

In [24]:
X_train[0]

'"Baker said he heard the voice of God telling him to kill the young carnival worker or “die immediately.”"\n\nBaker is no different from any other religious person.  Would it kill us to respect his religious views?'

In [0]:
avg_len = int(avg_len)

In [26]:
X_train_indices = sentences_to_indices(X_train,word_to_index, avg_len)
X_test_indices = sentences_to_indices(X_test,word_to_index, avg_len)

print(X_train_indices.shape)
print(X_test_indices.shape)

(230934, 49)
(57734, 49)


In [0]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    vocab_len = len(word_to_index) + 1            
    emb_dim = word_to_vec_map["cucumber"].shape[0]  
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
#    embedding_layer = Embedding(vocab_len,
#                             emb_dim,
#                             weights=[emb_matrix],
#                             trainable=False)
    
    return embedding_layer

In [28]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print(embedding_layer)

W0815 17:28:58.632283 140319923079040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 17:28:58.662808 140319923079040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0815 17:28:58.677361 140319923079040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0815 17:28:58.678355 140319923079040 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:181: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0815 17:28:58.679264 1403199230

In [0]:
def toxicity(input_shape, word_to_vec_map, word_to_index):
    sentence_indices = Input(input_shape, dtype='int32')
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices) 
    X = LSTM(256, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(256, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(1)(X)
    X =  Activation('sigmoid')(X)
    model = Model(inputs=sentence_indices, outputs=X)
    
    return model

In [43]:
model = toxicity((None,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_3 (Embedding)      (None, None, 100)         40000100  
_________________________________________________________________
lstm_3 (LSTM)                (None, None, 256)         365568    
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 256)         0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 256)               525312    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
__________

In [0]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [45]:
model.fit(X_train_indices, y_train,validation_data=(X_test_indices, y_test), epochs = 5,
          batch_size = 256, shuffle=True)

Train on 230934 samples, validate on 57734 samples
Epoch 1/5
230934/230934 [==============================] - 119s 514us/step - loss: 0.4529 - acc: 0.7860 - val_loss: 0.4090 - val_acc: 0.8141
Epoch 2/5
230934/230934 [==============================] - 117s 509us/step - loss: 0.3895 - acc: 0.8252 - val_loss: 0.3802 - val_acc: 0.8274
Epoch 3/5
230934/230934 [==============================] - 117s 509us/step - loss: 0.3699 - acc: 0.8350 - val_loss: 0.3733 - val_acc: 0.8358
Epoch 4/5
230934/230934 [==============================] - 116s 500us/step - loss: 0.3562 - acc: 0.8426 - val_loss: 0.3693 - val_acc: 0.8392
Epoch 5/5
230934/230934 [==============================] - 116s 504us/step - loss: 0.3433 - acc: 0.8493 - val_loss: 0.3580 - val_acc: 0.8416


In [46]:
model_json = model.to_json()
with open("/content/drive/My Drive/toxicity/lstm.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("/content/drive/My Drive/toxicity/lstm.h5")
print("Saved model to disk")

Saved model to disk


In [47]:
from keras.models import model_from_json
json_file = open('/content/drive/My Drive/toxicity/lstm.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights("/content/drive/My Drive/toxicity/lstm.h5")
print("Loaded model from disk")

loaded_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Loaded model from disk


In [0]:
def text_sentence_to_indices(text_sentence, word_to_index):
    tmp = text_sentence.translate(str.maketrans('','',string.punctuation)).lower().split()
    j = 0
    X_indices = np.zeros((1, len(tmp)))
    for w in tmp:
      try:
        X_indices[0, j] = word_to_index[w]
        j = j+1
        #print(w)
      except KeyError:
        #print(w)
        pass
            
    
    return X_indices

In [70]:
s = "I am learning NLP"
ind = text_sentence_to_indices(s,word_to_index)
print(ind)

[[185457.  52943. 219115. 262255.]]


In [71]:
loaded_model.predict(ind)

array([[0.22894216]], dtype=float32)

In [84]:
s = "What an asshole!"
ind = text_sentence_to_indices(s,word_to_index)
print(ind)

[[386307.  54273.  61650.]]


In [85]:
loaded_model.predict(ind)

array([[0.94075763]], dtype=float32)